# Performance Investigation of Hybrid Life-Cycle Assessment Matrix Calculations

This notebook is available online at this Zenodo Record: [`doi:10.5281/zenodo.14786979`](https://doi.org/10.5281/zenodo.14786979)

Note that this investigation was originally run in January 2025 run using a virutal environment with the following packages:

```
numpy==2.2.1
scipy==1.15.0
```

In [1]:
# scientific computing
import numpy as np
rng = np.random.default_rng(seed=42)
import pandas as pd
# data storage
import gzip
import pickle
# system libraries
import time

## Load Compressed Data from `pylcaio` Package

In [2]:
%%capture
path = '/Users/michaelweinold/data/HLCA/hybrid_system.pickle'
with gzip.open(path, 'rb') as pickle_file:
    picklefile = pickle.load(file=pickle_file)

## Build Hybrid Matrices

For the definition of the matrices in the output of the [`pylcaio` package](https://github.com/MaximeAgez/pylcaio/tree/master), see [this section of the source code](https://github.com/MaximeAgez/pylcaio/blob/505898a39144ebc53c109e485644e3ea055ae0ae/src/pylcaio.py#L46
).

The matrices are defined as follows:

| Symbol | Dimension | Units | Description |
| ------ | --------- | ----- | ----------- |
| $\mathbf{A}_P$ | $M \times M$ | [kg] ("physical") | technosphere matrix |
| $\mathbf{A}_S$ | $N \times N$ | [\$] ("monetary") | technical coefficient matrix |
| $\mathbf{C}_U$ | $N \times M$ | [\$/kg] |  upstream cut-off matrix |
| $\mathbf{B}_P$ | $R \times M$ | [CO₂/kg] ("environmental flow") | biosphere matrix |
| $\mathbf{B}_S$ | $P \times N$ | [CO₂/\$] ("environmental flow") | environmental satellite matrix |
| $\mathbf{Q}_P$ | $1 \times R$ | [°warming/CO₂] | characterization matrix process system |
| $\mathbf{Q}_S$ | $1 \times P$ | [°warming/CO₂] | characterization matrix sectoral system |

Note that for simplicity, here we assume that we have only a single environmental burden (global warming potential). Therefore the dimension of $\mathbf{Q}=1 \times ...$.

The hybrid matrices are then build such that:

\begin{align}
\mathbf{A}_H &= \begin{bmatrix}
\mathbf{A}_P & \mathbf{0} \\
\mathbf{C}_U & \mathbf{A}_S
\end{bmatrix} \\
\mathbf{A}_H &= [(M+N) \times (M+N)]
\end{align}

and

\begin{align}
\mathbf{B}_H &= \begin{bmatrix}
\mathbf{B}_P & 0 \\
0 & \mathbf{B}_S
\end{bmatrix} \\
\mathbf{B}_H &= [(R+P) \times (M+N)]
\end{align}

and

\begin{align}
\mathbf{Q}_H &= \begin{bmatrix}
\mathbf{Q}_P & \mathbf{Q}_S
\end{bmatrix} \\
\mathbf{Q}_H &= [1 \times (R+P)]
\end{align}

Which gives the following equation:

\begin{align}
e &= \mathbf{Q}_H \cdot \mathbf{B}_H \cdot (\mathbf{A}_H)^{-1} \cdot \vec{f} \\
[1 \times 1] &= [1 \times (R+P)] \cdot [(R+P) \times (M+N)] \cdot [(M+N) \times 1]
\end{align}

In [3]:
A_P = picklefile['A_ff'].todense().A
A_S = picklefile['A_io'].todense().A
C_U = picklefile['A_io_f'].todense().A
A_H = np.block(
    [
        [np.eye(A_P.shape[0]) - A_P, np.zeros((A_P.shape[0], A_S.shape[0]))],
        [C_U, np.eye(A_S.shape[0]) - A_S]
    ]
)

B_S = picklefile['F_io'].todense().A
B_P = picklefile['F_f'].todense().A
B_H = np.block(
    [
        [B_P, np.zeros((B_P.shape[0], B_S.shape[1]))],
        [np.zeros((B_S.shape[0], B_P.shape[1])), B_S]
    ]
)

Q_P_climate = picklefile['C_f'].todense().A[0,:]
Q_S_climate = picklefile['C_io'].todense().A[0,:]
Q_H_climate = np.atleast_2d(np.concatenate((Q_P_climate, Q_S_climate), axis=0))

In [5]:
density_A_P = np.count_nonzero(A_P) / A_P.size
density_A_P

0.0005447512287780848

In [6]:
density_A_S = np.count_nonzero(A_S) / A_S.size
density_A_S

0.3530680862140775

In [7]:
density_A_H = np.count_nonzero(A_H) / A_H.size
density_A_H

0.044552485176151575

In [8]:
def generate_final_demand_vector(
    number_of_sectors: int,
    sector_index: int,
    demand_amount: float
) -> np.ndarray:
    """
    Generates a final demand vector with a given demand amount for a given sector and 0 for all other sectors.

    Parameters
    ----------
    number_of_sectors : int
        Number of sectors in the process database (number of rows or columns in the technosphere matrix).
    sector_index : int
        Index of the sector for which the final demand vector should be generated.
    demand_amount : float
        Demand amount for the given sector.

    Returns
    -------
    np.ndarray
        Final demand vector with a given demand amount for a given sector and 0 for all other sectors.
    """
    f_vector = np.zeros(number_of_sectors)
    f_vector[sector_index] = demand_amount
    return f_vector

In [16]:
f_P = generate_final_demand_vector(
    number_of_sectors=A_P.shape[0],
    sector_index=0,
    demand_amount=1.0
)
f_H = generate_final_demand_vector(
    number_of_sectors=A_H.shape[0],
    sector_index=1111,
    demand_amount=1.0
)

In [14]:
A_P

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(21255, 21255))

In [ ]:
%%time
np.linalg.solve(A_H, f_H)

CPU times: user 4min 34s, sys: 8.38 s, total: 4min 43s
Wall time: 2min 36s


array([ 1.02459975e-13,  9.44899116e-15,  1.06404106e-14, ...,
       -2.62170889e-07, -2.77594759e-07,  0.00000000e+00], shape=(31055,))

In [19]:
np.linalg.inv(A_P)

LinAlgError: Singular matrix

In [4]:
print(f'dim(A_P)={A_P.shape}')
print(f'dim(A_S)={A_S.shape}')
print(f'dim(C_U)={C_U.shape}')
print(f'dim(A_H)={A_H.shape}')
print(f'dim(B_P)={B_P.shape}')
print(f'dim(B_S)={B_S.shape}')
print(f'dim(B_H)={B_H.shape}')
print(f'dim(Q_P_climate)={Q_P_climate.shape}')
print(f'dim(Q_S_climate)={Q_S_climate.shape}')
print(f'dim(Q_H_climate)={Q_H_climate.shape}')

dim(A_P)=(21255, 21255)
dim(A_S)=(9800, 9800)
dim(C_U)=(9800, 21255)
dim(A_H)=(31055, 31055)
dim(B_P)=(4709, 21255)
dim(B_S)=(716, 9800)
dim(B_H)=(5425, 31055)
dim(Q_P_climate)=(4709,)
dim(Q_S_climate)=(716,)
dim(Q_H_climate)=(1, 5425)


## Prepare Random Sample of Ecoinvent Processes

Every Ecoinvent process has an industry classification code according to the International Standard Industrial Classification of All Economic Activities (ISIC). We use the highest-level classification structure of the 21 (A-U) ISIC "sections" to group Ecoinvent processes (see ["Classification Structure"](https://unstats.un.org/unsd/classifications/Family/Detail/27)).

In [5]:
dict_isic_letters_and_numbers = {
    'A': ['01', '02', '03'],
    'B': ['05', '06', '07', '08', '09'],
    'C': [
        '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
        '31', '32', '33'
    ],
    'D': ['35'],
    'E': ['36', '37', '38', '39'],
    'F': ['41', '42', '43'],
    'G': ['45', '46', '47'],
    'H': ['49', '50', '51', '52', '53'],
    'I': ['55', '56'],
    'J': ['58', '59', '60', '61', '62', '63'],
    'K': ['64', '65', '66'],
    'L': ['68'],
    'M': ['69', '70', '71', '72', '73', '74', '75'],
    'N': ['77', '78', '79', '80', '81', '82'],
    'O': ['84'],
    'P': ['85'],
    'Q': ['86', '87', '88'],
    'R': ['90', '91', '92', '93'],
    'S': ['94', '95', '96'],
    'T': ['97', '98'],
    'U': ['99']
}
list_process_metadata_isic = [i for i in picklefile['PRO_f']['ISIC'].values()]
list_process_metadata_isic_numbers = [str(string)[:2] for string in list_process_metadata_isic]

In [6]:
def get_sample_process_indices_from_ecoinvent_per_isic_letter(
    dict_isic_letters_and_numbers: dict,
    list_process_metadata_isic_numbers: list,
    number_of_indices_per_isic_letter: int,
) -> dict:
    """
    Gets a random sample of process indices from the ecoinvent database for each ISIC letter (=sections).

    If the number of processes for a given ISIC letter is less than the number of indices to be sampled,
    the function will sample all available processes.

    Parameters
    ----------
    dict_isic_letters_and_numbers : dict
        Dictionary with ISIC letters (=sections) as keys and lists of ISIC numbers (=divisions) as values.
        For example: {'A': ['01', '02', '03'], 'B': ['05', '06', '07', '08', '09'], ...}
    list_process_metadata_isic_numbers : list
        List of ISIC numbers for each process in the ecoinvent database.
        For example: ['01', '01', '02', '02', '02', '03', ...]
    number_of_indices_per_isic_letter : int
        Number of indices to be sampled for each ISIC letter.

    Returns
    -------
    dict
        Dictionary with ISIC letters as keys and lists of sampled process indices as values.
        For example: {'A': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'B': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], ...}
    """
    
    dict_isic_letters_and_indices = {}

    for isic_letter in dict_isic_letters_and_numbers.keys():
        list_of_process_indices = []
        for isic_number in dict_isic_letters_and_numbers[isic_letter]:
            list_of_process_indices += [index for index, element in enumerate(list_process_metadata_isic_numbers) if element == isic_number]
        dict_isic_letters_and_indices[isic_letter] = list_of_process_indices

    dict_return = {}

    for isic_letter, isic_numbers in dict_isic_letters_and_indices.items():
        sample_size = number_of_indices_per_isic_letter
        if dict_isic_letters_and_indices[isic_letter] == []:
            continue
        if len(isic_numbers) < number_of_indices_per_isic_letter:
            sample_size = len(isic_numbers)
        dict_return[isic_letter] = rng.choice(
            a=dict_isic_letters_and_indices[isic_letter],
            size=sample_size,
            replace=False
        ).tolist()
        
    return dict_return

In [7]:
dict_sample_process_indices = get_sample_process_indices_from_ecoinvent_per_isic_letter(
    dict_isic_letters_and_numbers=dict_isic_letters_and_numbers,
    list_process_metadata_isic_numbers=list_process_metadata_isic_numbers,
    number_of_indices_per_isic_letter=10
)
dict_sample_process_indices

{'A': [234, 2109, 243, 1785, 1197, 1181, 1905, 257, 550, 2343],
 'B': [3277, 3014, 2990, 3182, 3309, 2860, 3045, 3080, 3115, 3378],
 'C': [8679, 9109, 5505, 6773, 6089, 4670, 8162, 10126, 10705, 3942],
 'D': [15306, 12787, 14297, 12402, 14016, 14288, 12586, 15029, 11624, 15100],
 'E': [18157, 18666, 16330, 18035, 19078, 17884, 17140, 16076, 17629, 16727],
 'F': [19942, 19578, 19319, 19498, 19533, 19442, 19900, 20083, 20053, 20059],
 'G': [20249, 20263, 20256, 20243, 20255, 20245, 20258, 20254, 20271, 20274],
 'H': [20471, 20614, 20446, 20831, 20432, 20720, 20588, 20522, 20397, 20293],
 'I': [20851, 20846, 20853, 20850, 20844, 20843, 20849, 20852, 20847, 20848],
 'J': [20856, 20868, 20870, 20871, 20854, 20864, 20862, 20867, 20860, 20861],
 'M': [20875, 20874, 20876, 20877, 20873, 20872],
 'N': [20905, 20962, 20980, 20914, 20968, 20925, 20949, 20906, 20958, 20928],
 'S': [20989, 20990, 20986, 20988, 20987]}

## Calculations with Hybrid LCA Matrix

The below code simply implements the governing equation of hybrid life-cycle assessment:

\begin{align}
e &= \mathbf{Q}_H \cdot \mathbf{B}_H \cdot (\mathbf{A}_H)^{-1} \cdot \vec{f} \\
[1 \times 1] &= [1 \times (R+P)] \cdot [(R+P) \times (M+N)] \cdot [(M+N) \times 1]
\end{align}

In [15]:
def generate_final_demand_vector(
    number_of_sectors: int,
    sector_index: int,
    demand_amount: float
) -> np.ndarray:
    """
    Generates a final demand vector with a given demand amount for a given sector and 0 for all other sectors.

    Parameters
    ----------
    number_of_sectors : int
        Number of sectors in the process database (number of rows or columns in the technosphere matrix).
    sector_index : int
        Index of the sector for which the final demand vector should be generated.
    demand_amount : float
        Demand amount for the given sector.

    Returns
    -------
    np.ndarray
        Final demand vector with a given demand amount for a given sector and 0 for all other sectors.
    """
    f_vector = np.zeros(number_of_sectors)
    f_vector[sector_index] = demand_amount
    return f_vector
 

def compute_environmental_burden(
    A_H: np.ndarray,
    B_H: np.ndarray,
    Q_H_climate: np.ndarray,
    sector_index: int,
) -> tuple[float, float]:
    """
    Computes the environmental burden for a given sector index.

    See Also
    --------
    - [Eqn. (2.35) and (8.27) in Heijungs & Suh (2002)](https://doi.org/10.1007/978-94-015-9900-9)

    Parameters
    ----------
    A_H : np.ndarray
        Hybrid A-matrix.
    B_H : np.ndarray
        Hybrid biosphere/environmental satellite matrix.
    Q_H_climate : np.ndarray
        Hybrid characterization matrix.
    sector_index : int
        Index of the sector for which the environmental burden should be computed.

    Returns
    -------
    tuple[float, float]
        Tuple with the environmental burden and the computation time.
    """
    f_vector_H = generate_final_demand_vector(
        number_of_sectors=A_H.shape[0],
        sector_index=sector_index,
        demand_amount=1
    )
    start = time.time()
    vec_intermediate_demand = np.linalg.solve(A_H, f_vector_H)
    vec_environmental_flows = np.dot(B_H, vec_intermediate_demand)
    scalar_environmental_burden = np.dot(Q_H_climate, vec_environmental_flows)
    end = time.time()
    computation_time = end - start
    return scalar_environmental_burden, computation_time

Before starting calculations, ensure that your local NumPy is built against a fast [BLAS library](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) (e.g., Intel MKL, OpenBLAS, or Apple Accelerate). Note that on a 2021 MacBook Pro (M1 Max CPU) with NumPy v2.2.1 [built against Apple Accelerate](https://numpy.org/doc/2.0/release/1.21.0-notes.html#enable-accelerate-framework), one computation takes approximately 130-190 seconds.

In [13]:
def run_computation(
    dict_sample_process_indices: dict,
    number_of_iterations: int,
) -> None:
    """
    Computes the environmental burden and computation time for a given sector index.
    For every sector, the computation is repeated a given number of times to check for numerical stability.

    For every ISIC letter (=section), the function stores the results in a pandas DataFrame and saves it as a pickle file.

    The DataFrame is of the form:

    | sector_index | computation_times | environmental_burden | cv  |
    |--------------|-------------------|----------------------|-----|
    | 1452         | [150, 155, 148]   | [0.1, 0.1, 0.1]      | 0.0 |
    | 2254         | [156, 171, 180]   | [0.3, 0.3, 0.3]      | 0.0 |
    | ...          | ...               | ...                  | ... |

    Parameters
    ----------
    dict_sample_process_indices : dict
        Dictionary with ISIC letters as keys and lists of sampled process indices as values.
        For example: {'A': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'B': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], ...}
    """
    for isic_letter in dict_sample_process_indices.keys():
        print('Now processing ISIC section:', isic_letter)
        list_of_list_computation_times = []
        list_of_list_results = []
        list_cv = []
        for sector_index in dict_sample_process_indices[isic_letter]:
            list_results = []
            list_computation_times = []
            print('Index of sector:', sector_index)
            for _ in range(0, number_of_iterations):
                result = compute_environmental_burden(
                    A_H=A_H,
                    B_H=B_H,
                    Q_H_climate=Q_H_climate,
                    sector_index=sector_index
                )
                list_results.append(result[0])
                print(f'Computing environmental burden took {result[1]} seconds.')
                list_computation_times.append(result[1])
            
            std_result = np.std(list_results, ddof=1)
            mean_result = np.mean(list_results)
            list_cv.append(std_result / mean_result)
            list_of_list_results.append(list_results)
            list_of_list_computation_times.append(list_computation_times)

        df_results_isic_section = pd.DataFrame(
            {
                'sector_index': dict_sample_process_indices[isic_letter],
                'computation_times': list_of_list_computation_times,
                'environmental_burden': list_of_list_results,
                'cv': list_cv
            }
        )
        with open(f'results_section_{isic_letter}.pkl', 'wb') as f:
            pickle.dump(df_results_isic_section, f)

In [ ]:
run_computation(
    dict_sample_process_indices=dict_sample_process_indices,
    number_of_iterations=5
)